In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [15]:
url_inicial = 'https://revistapediatria.org/rp/issue/archive/2'
url_root = 'https://revistapediatria.org/rp/issue/archive/2'
r=requests.get(url_inicial)

In [16]:
soup = BeautifulSoup(r.text, 'html.parser')

In [17]:
box = soup.find('div', class_="issues media-list")
volumen=box.findAll('div', class_='media-body')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistapediatria.org/rp/issue/view/6',
 'https://revistapediatria.org/rp/issue/view/7',
 'https://revistapediatria.org/rp/issue/view/8',
 'https://revistapediatria.org/rp/issue/view/10']

In [18]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [19]:
vol2

['https://revistapediatria.org/rp/article/view/36',
 'https://revistapediatria.org/rp/article/view/37',
 'https://revistapediatria.org/rp/article/view/38',
 'https://revistapediatria.org/rp/article/view/39',
 'https://revistapediatria.org/rp/article/view/41',
 'https://revistapediatria.org/rp/article/view/42',
 'https://revistapediatria.org/rp/article/view/43',
 'https://revistapediatria.org/rp/article/view/40',
 'https://revistapediatria.org/rp/article/view/44',
 'https://revistapediatria.org/rp/article/view/45',
 'https://revistapediatria.org/rp/article/view/46',
 'https://revistapediatria.org/rp/article/view/47',
 'https://revistapediatria.org/rp/article/view/56',
 'https://revistapediatria.org/rp/article/view/59',
 'https://revistapediatria.org/rp/article/view/58']

In [20]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [21]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://revistapediatria.org/rp/issue/archive/2


In [22]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

15

In [23]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [24]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Pediatría'
    a='III. Medicina y Ciencias de la Salud'
    tem='Pediatría'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-summary').get_text(strip=True)
        content_book['Resumen']=resu.replace('Resumen','')
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [25]:
list_scraper=list_scraper[0:15]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14


In [26]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Abstract,Link inicial,Link articulo
0,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Opciones de tratamiento del cólico del lactante,DOI: 10.1016/j.rcpe.2015.10.001IntroducciónEl ...,None,https://revistapediatria.org/rp/article/view/36,https://revistapediatria.org/rp/article/view/3...
1,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Alimentos derivados de cultivos genéticamente ...,DOI: 10.1016/j.rcpe.2015.09.001Desde el año 19...,None,https://revistapediatria.org/rp/article/view/37,https://revistapediatria.org/rp/article/view/3...
2,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Diagnóstico neonatal de hidrometrocolpos,DOI: 10.1016/j.rcpe.2015.10.002Se presenta el ...,None,https://revistapediatria.org/rp/article/view/38,https://revistapediatria.org/rp/article/view/3...
3,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Enfermedad granulomatosa crónica: reporte de caso,DOI: 10.1016/j.rcpe.2015.10.003La Enfermedad G...,None,https://revistapediatria.org/rp/article/view/39,https://revistapediatria.org/rp/article/view/3...
4,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Guía de Práctica Clínica de la enfermedad diar...,DOI: 10.1016/j.rcpe.2015.07.005Como parte del ...,None,https://revistapediatria.org/rp/article/view/41,https://revistapediatria.org/rp/article/view/4...
5,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Desarrollo de la tamización neonatal en Colomb...,DOI: 10.1016/j.rcpe.2015.07.006AntecedentesEn ...,None,https://revistapediatria.org/rp/article/view/42,https://revistapediatria.org/rp/article/view/4...
6,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Caracterización de los pacientes pediátricos c...,DOI: 10.1016/j.rcpe.2015.07.007AntecedentesLa ...,None,https://revistapediatria.org/rp/article/view/43,https://revistapediatria.org/rp/article/view/4...
7,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Editorial Guía de práctica clínica para la pre...,DOI: 10.1016/j.rcpe.2015.07.004La enfermedad d...,None,https://revistapediatria.org/rp/article/view/40,https://revistapediatria.org/rp/article/view/4...
8,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Microbiota de la glándula mamaria,DOI: 10.1016/j.rcpe.2015.07.001Revisar y descr...,None,https://revistapediatria.org/rp/article/view/44,https://revistapediatria.org/rp/article/view/4...
9,Revista Mexicana de Pediatría,III. Medicina y Ciencias de la Salud,Pediatría,Síndrome de muerte súbita y otras muertes infa...,DOI: 10.1016/j.rcpe.2015.07.002ObjetivosSe pre...,None,https://revistapediatria.org/rp/article/view/45,https://revistapediatria.org/rp/article/view/4...


In [27]:
df_catalogo.to_csv('Revista091.02.csv', index=False)